## Import libraries

In [2]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
import numpy as np
import pandas as pd
import sys
sys.path.append('..')
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from cuml import RandomForestRegressor as cuRF
import xgboost as xgb
import math
import time
import re
from scripts.clustering_utils import read_clusters_from_file, cluster_predict, cluster_predict_ew, aggregate_cluster_predictions, dataframe_by_cluster, split_into_clusters, select_past_timesteps

In [3]:
n_clusters = 16

## Read Dataset

In [8]:
df = pd.read_csv("../../data/loureiro_45houses.csv")
#df_index = pd.DataFrame(pd.date_range('2019-01-01', periods=35136, freq='15T'))
#df = pd.concat([pd.DataFrame(df_index).rename(columns={0: "Time"}),df],axis=1)
df

,Time,Energy_1,Energy_2,Energy_9,Energy_11,Energy_15,Energy_16,Energy_17,Energy_29,Energy_30,...,Energy_139,Energy_142,Energy_146,Energy_148,Energy_151,Energy_159,Energy_160,Energy_161,Energy_164,Energy_172
0,2022-05-05 12:00:00,0.068,0.018,0.024,0.114,0.164,0.072,0.055,0.011,0.006,...,0.083,0.084,0.068,0.060,0.042,0.055,0.069,0.010,0.004,0.008
1,2022-05-05 12:15:00,0.070,0.018,0.024,0.190,0.135,0.058,0.060,0.002,0.006,...,0.053,0.004,0.048,0.078,0.053,0.055,0.063,0.010,0.003,0.056
2,2022-05-05 12:30:00,0.122,0.018,0.024,0.180,0.121,0.011,0.096,0.010,0.006,...,0.035,0.004,0.062,0.061,0.057,0.055,0.049,0.010,0.004,0.033
3,2022-05-05 12:45:00,0.132,0.018,0.024,0.167,0.138,0.010,0.064,0.004,0.006,...,0.082,0.004,0.056,0.125,0.034,0.055,0.097,0.010,0.004,0.062
4,2022-05-05 13:00:00,0.125,0.018,0.024,0.133,0.104,0.011,0.072,0.002,0.006,...,0.076,0.004,0.034,0.070,0.038,0.055,0.075,0.010,0.004,0.024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38011,2023-06-05 10:45:00,0.094,0.032,0.074,0.203,0.108,0.002,0.124,0.012,0.013,...,0.063,0.004,0.063,0.014,0.042,0.168,0.397,0.019,0.003,0.028
38012,2023-06-05 11:00:00,0.094,0.032,0.074,0.203,0.108,0.002,0.124,0.012,0.013,...,0.063,0.004,0.063,0.014,0.042,0.168,0.397,0.019,0.003,0.028
38013,2023-06-05 11:15:00,0.094,0.032,0.074,0.203,0.108,0.002,0.124,0.012,0.013,...,0.063,0.004,0.063,0.014,0.042,0.168,0.397,0.019,0.003,0.028
38014,2023-06-05 11:30:00,0.094,0.032,0.074,0.203,0.108,0.002,0.124,0.012,0.013,...,0.063,0.004,0.063,0.014,0.042,0.168,0.397,0.019,0.003,0.028


In [4]:
names = ["Linear Regression", "XGBoost", "Random Forest"]

In [10]:
## Complete script
sourceFile = open("../../gpu_logs/new_clusters_loureiro_dtwnoew.txt", "w")
all_clusters = read_clusters_from_file("new_clusters_loureiro_dtw.txt", n_clusters)
df_houses = df.drop("Time", axis=1)
estimators = [LinearRegression(), xgb.XGBRegressor(tree_method='gpu_hist', seed=0, colsample_bytree=0.7, learning_rate=0.1, max_depth=10, n_estimators=500), cuRF()]
all_cluster_preds = []
n_c = 2
for c in all_clusters[5:8]:
    print("Number of Clusters: {}".format(n_c))
    print("Number of Clusters: {}".format(n_c), file=sourceFile)
    cluster_houses = split_into_clusters(c)
    i = 0
    pred_list = []
    clusters_preds, clusters_actuals = [], []
    for clust in cluster_houses:
        clust = [df_houses.columns[d] for d in clust]
        print("Cluster {}".format(i))
        i += 1
        df_cluster = select_past_timesteps("../../data/loureiro_weather_45houses_selected.csv", clust)
        c_pred, c_test = cluster_predict(df_cluster, estimators=estimators, names=names)
        clusters_preds.append(c_pred)
        clusters_actuals.append(c_test)
    n_c += 1
    agg_pred = aggregate_cluster_predictions(clusters_preds, clusters_actuals, names, sourceFile)
    all_cluster_preds.append(agg_pred)
sourceFile.close()

Number of Clusters: 2
Cluster 0
Elapsed time training and predicting: 3.7510 s
Elapsed time training and predicting: 31.4235 s
Elapsed time training and predicting: 16.8859 s
Cluster 1
Elapsed time training and predicting: 4.8404 s
Elapsed time training and predicting: 29.6804 s
Elapsed time training and predicting: 19.8527 s
Cluster 2
Elapsed time training and predicting: 2.8166 s
Elapsed time training and predicting: 28.7565 s
Elapsed time training and predicting: 14.3248 s
Cluster 3
Elapsed time training and predicting: 0.3000 s
Elapsed time training and predicting: 20.9260 s
Elapsed time training and predicting: 4.8959 s
Cluster 4
Elapsed time training and predicting: 0.1383 s
Elapsed time training and predicting: 15.9388 s
Elapsed time training and predicting: 3.9883 s
Cluster 5
Elapsed time training and predicting: 0.2924 s
Elapsed time training and predicting: 22.1842 s
Elapsed time training and predicting: 5.0483 s
Cluster 6
Elapsed time training and predicting: 0.3227 s
Elapse

## Best predictions

In [29]:
with open('../../gpu_logs/new_clusters_loureiro_dtw.txt', 'r') as f:
    l = f.readlines()
    lines = [s for s in l if re.search(r'\w+', s)]

In [30]:
def get_cluster_metrics(lines):
    cursor = 0
    model_names, mse, wape, r2, number_clust = [], [], [], [], []
    for i in lines:
        if i.startswith("Number"):
            number_clust.append(int(i[-3:-1].strip()))
        else:
            if cursor == 0:
                model_names.append(i[:-1])
                cursor += 1
            elif cursor == 1:
                mse.append(i.split(":")[1][:-1])
                cursor += 1
            elif cursor == 2:
                wape.append(i.split(":")[1][:-1])
                cursor += 1
            elif cursor == 3:
                r2.append(i.split(":")[1][:-1])
                cursor = 0
    mse = list(map(float, mse))
    wape = list(map(float, wape))
    wape = list(map(lambda x: x/100, wape))
    r2 = list(map(float, r2))
    return model_names, mse, wape, r2, number_clust

In [31]:
def best_number_of_cluster(lines):
    model_names, mse, wape, r2, number_clust = get_cluster_metrics(lines)
    min_mse, min_index_mse, mod_index_mse = min(mse), int(np.argmin(mse)/3), np.argmin(mse)%3
    min_wape, min_index_wape, mod_index_wape = min(wape), int(np.argmin(wape)/3), np.argmin(wape)%3
    max_r2, max_index_r2, mod_index_r2 = max(r2), int(np.argmax(r2)/3), np.argmax(r2)%3
    return min_mse, min_wape, max_r2, number_clust[max_index_r2], model_names[mod_index_r2]

In [32]:
best_mse, best_wape, best_r2, best_n_clust, best_alg = best_number_of_cluster(lines)
best_mse, best_wape, best_r2, best_n_clust, best_alg

(0.0582, 0.3389, 0.6595, 3, 'XGBoost')